In [1]:
chars = ""
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
   text = f.read()
   chars = sorted(list(set(text)))
   
vocab_size = len(chars)

In [2]:
# tokenize the chars
string_to_int = { ch: i for i, ch in enumerate(chars)} # create a dictionary of the characters
int_to_string = {i:ch for i, ch in enumerate(chars)} # create a reverse dictionary of the characters
encode = lambda words: [string_to_int[char] for char in words]
decode = lambda chars: ''.join([int_to_string[char] for char in chars])

encode_hello = encode('hello')
print(encode_hello)
decoded_hello = decode(encode_hello)
print(decoded_hello)


[61, 58, 65, 65, 68]
hello


In [3]:
import torch

### Using batches to accelerate training process


In [4]:
# check if cuda is available in our hardware
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 64 # length of the sequence
batch_size = 128 # number of blocks we are processing in parallel
max_iters = 3000 # number of iterations
learning_rate = 3e-4 # learning rate equivalent to 0.0003
eval_iters = 100
n_embd = 384 # number of embeddings
n_head = 8 # number of heads
dropout = 0.2 # dropout rate
n_layer = 8 # number of layers

cpu


In [5]:
data = torch.tensor(encode(text), dtype=torch.long) # convert the text to a tensor
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


## Validation and Training Split


In [6]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y  = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
import torch.nn as nn
from torch.nn import functional as F 

class Head(nn.Module):
    """ one head of seld-attention """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch , time-step , channels)
        B,T,C = x.shape
        k = self.key(x) # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim = -1 ) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B, T, hs)
        out = wei @ v 
        return out

class MultiheadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.num_heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.num_heads], dim=-1) # (B, T, F)
        out  = self.dropout(self.proj(out))
        return out
    
    
class FeedForward(nn.Module):
    """ A simple linear layer followed by a non-linearity """
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like 
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiheadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocab_size, n_embd)
        self.positional_embeddings_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer form
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets = None):
        B, T = index.shape
        
        
        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embeddings_table(index) # (B, T, C)
        pos_emb = self.positional_embeddings_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the functionos
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # becomes (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # becomes (B,1)
            # append sampled index to the running sequence
            index = torch.cat([index, index_next], dim=1) # becomes (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, test loss: {losses['test']:.4f}")
    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the  loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


step: 0, train loss: 4.5286, test loss: 4.5244
step: 100, train loss: 2.3200, test loss: 2.3902
step: 200, train loss: 1.8481, test loss: 1.9763
step: 300, train loss: 1.6063, test loss: 1.7646


KeyboardInterrupt: 

**_Need to familiarize _**

1. Mean Squared Error(MSE)
2. Gradient Descent (GD)
3. Momentum
4. RMSprop
5. Adam
6. AdamW


In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

IndexError: index out of range in self